In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

In [5]:
import torch
import numpy as np

In [51]:
x = np.random.normal(size = (2, 32, 100))
x_tf = tf.convert_to_tensor(x)
x_pt = torch.from_numpy(x)

In [52]:
step = 2
size = 3
x_pt.unfold(1, size, step).shape

torch.Size([2, 15, 100, 3])

In [50]:
2 * 32 * 100, 2 * 15 * 100 * 3

(6400, 9000)

In [27]:
# tf.cast((sizedim - size) / step + 1, tf.int32)

In [36]:
tf.cast((sizedim - size) / step + 1, tf.int32)

<tf.Tensor: id=119, shape=(), dtype=int32, numpy=30>

In [49]:
# unfold(dimension, size, step)
# (sizedim - size) / step + 1

sizedim = tf.shape(x_tf)[1]
tf.reshape(x_tf, [2, tf.cast((sizedim - size) / step + 1, tf.int32), 100, size])

InvalidArgumentError: Input to reshape is a tensor with 6400 values, but the requested shape has 9000 [Op:Reshape]

In [17]:
x_pt.unfold(2, 2, 2).numpy() == tf.reshape(x_tf, [2, 32, 50, 2]).numpy()

array([[[[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        ...,

        [[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True],
         ...,
         [ True,  True],
         [ True,  True],
         [ True, 

In [14]:
tf.reshape(x_tf, [2, 32, 50, 2])

<tf.Tensor: id=2, shape=(2, 32, 50, 2), dtype=float64, numpy=
array([[[[-1.62255594,  0.53987   ],
         [-0.62466927, -0.64393786],
         [ 1.25499363,  0.82886115],
         ...,
         [-1.05256185,  0.79822693],
         [-0.17659151,  1.15927588],
         [ 0.12720288,  0.82986435]],

        [[ 1.9291868 ,  0.23002664],
         [ 0.30287069,  0.15476743],
         [-0.16516713,  1.01026436],
         ...,
         [ 0.84398919, -1.03177384],
         [ 0.0468182 , -0.15791798],
         [-0.37574049, -0.13320144]],

        [[ 0.32845026,  0.34813471],
         [-0.18065284,  1.30400975],
         [ 0.5326798 , -0.29877265],
         ...,
         [ 1.20150123, -0.10677512],
         [-1.63258374,  0.54805731],
         [-0.69695462,  0.45821976]],

        ...,

        [[ 0.1074114 , -0.28028513],
         [-0.39293433, -0.12731034],
         [ 1.48159904,  0.79063345],
         ...,
         [ 0.75273727,  0.44813506],
         [ 0.41117212,  1.06264252],
         [ 

In [ ]:
from malaya_speech.train.model.univnet import layer